# Sesión 15: Análisis No Supervisado

## **Análisis Cluster: Kmeans**

La data de los diferentes paises del mundo tiene como variables los siguientes indicadores.
<table><tr><th>country</th><td>Nombre del país</td></tr>
<tr><th>region</th><td>Continente al que pertenece</td></tr>
<tr><th>tfr</th><td>Ratio de fertilidad</td></tr>
<tr><th>contraception</th><td>% de la población que usa anticonceptivos</td></tr>
<tr><th>educationMale</th><td>% de la población masculina educada</td></tr>
<tr><th>educationFemale</th><td>% de la población femenina educada</td></tr>
<tr><th>lifeMale</th><td>Esperanza de vida de los hombres</td></tr>
<tr><th>lifeFemale</th><td>Esperanza de vida de las mujeres</td></tr>
<tr><th>infantMortality</th><td>Ratio de mortalidad infantil</td></tr>
<tr><th>GDPperCapita</th><td>PBI</td></tr>
<tr><th>economicActivityMale</th><td>Hombres en la PEA</td></tr>
<tr><th>economicActivityFemale</th><td>Mujeres en la PEA</td></tr>
<tr><th>illiteracyMale</th><td>Ratio de analfabetismo en hombres</td></tr>
<tr><th>illiteracyFemale</th><td>Ratio de analfabetismo en mujeres</td></tr>
</table>

### Carga de módulos necesarios

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances_argmin_min

import scipy.cluster.hierarchy as shc

### Lectura

In [ ]:
#Cargando el dataset
data = pd.read_csv('data/UN.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
# data.columns
data.dtypes

% de missings por variables

In [ ]:
print('% de poblamiento de las variables\n')
print((data.isnull().sum()/data.shape[0])*100)

Vamos a considerar a todas las variables que superen el 50% de información completa.

In [ ]:
df = data[['tfr','contraception','lifeMale', 'lifeFemale', 'infantMortality', 'GDPperCapita','economicActivityMale','economicActivityFemale','illiteracyMale','illiteracyFemale']]
df = df.dropna(how='any')

In [ ]:
print('database original:', data.shape)
print('database reducida:', df.shape)

In [ ]:
df.head()

### Modelamiento clusters

In [ ]:
# Seleccionando nuestro set de datos
X = df

In [ ]:
# Generando nuestro primer cluster
kmeans = KMeans(n_clusters=4)

In [ ]:
kmeans_model = kmeans.fit(X)

In [ ]:
# Generamos la predicción de nuestro cluster
cluster = kmeans_model.predict(X)
len(cluster)

In [ ]:
# Obtenemos los centros
centers = kmeans_model.cluster_centers_
print(centers)
print(centers.shape)

In [ ]:
# Obtenemos los indicadores de clustering:

# Suma de las distancias al cuadrado de las muestras al centro de agrupación más cercano.
print('Inercia: '+str(kmeans_model.inertia_))

# El gráfico de silueta muestra una medida de qué tan cerca está cada punto de un grupo a los puntos en los grupos vecinos
print('Silueta: '+str(silhouette_score(X, cluster, metric='euclidean')))

# Entendiendo la silueta
# +1 indica que la muestra está muy lejos de los grupos vecinos
#  0 indica que la muestra está en o muy cerca del límite de decisión entre dos grupos vecinos
# -1 indica que la muestra está asignada al grupo incorrecto.

In [ ]:
# Graficamos nuestro cluster
v1 = df['infantMortality'].values
v2 = df['contraception'].values

colores=['red','blue','green','yellow']
asignar=[]

for row in cluster:
    asignar.append(colores[row])

In [ ]:
plt.scatter(v1, v2, c=asignar, marker='*')
plt.show()

### Modelamiento el número de clusters optimos

In [ ]:
score = []

for i in range(1, 20):
    kmeans = KMeans(n_clusters=i)
    kmeans_model = kmeans.fit(X)
    score.append(kmeans_model.inertia_)

In [ ]:
plt.plot(range(1, 20), score, marker='o')
plt.xlabel('Número de Clúster')
plt.ylabel('Score')
plt.title('Elbow method')
plt.show()

### Calculamos nuestro cluster final

In [ ]:
# Generando nuestro primer cluster
kmeans = KMeans(n_clusters=3)
kmeans_model = kmeans.fit(X)

In [ ]:
# Generamos la predicción de nuestro cluster
cluster = kmeans_model.predict(X)
len(cluster)

In [ ]:
# Obtenemos los indicadores de clustering:
print('Inercia: '+str(kmeans_model.inertia_)) 
print('Silueta: '+str(silhouette_score(X, cluster, metric='euclidean'))) 

### Agregando a nuestro set de datos

In [ ]:
df['cluster'] = cluster

In [ ]:
df.head()

In [ ]:
df['cluster'].value_counts()

In [ ]:
# tabla de frecuencia relativa
round(100 * df['cluster'].value_counts()/ len(df['cluster']),2)

### Un ejemplo para graficar clusters

In [ ]:
np.random.seed(42)
d0 = np.random.randn(150, 2)
d1 = np.random.randn(150, 2)+[0,3]
d2 = np.random.randn(150, 2)+3

features = np.concatenate([d0, d1, d2])

plt.scatter(features[:, 0], features[:, 1])
plt.show()

#### 2 Clusters

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(features)
pred_2 = kmeans.predict(features)

In [ ]:
plt.scatter(features[(pred_2 == 0), 0], features[(pred_2 == 0), 1], color = 'r')
plt.scatter(features[(pred_2 == 1), 0], features[(pred_2 == 1), 1], color = 'b')
plt.show()

#### 3 Clusters

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(features)
pred_2 = kmeans.predict(features)

In [ ]:
centers = kmeans.cluster_centers_
centers

In [ ]:
plt.scatter(features[(pred_2 == 0), 0], features[(pred_2 == 0), 1], color = 'r')
plt.scatter(features[(pred_2 == 1), 0], features[(pred_2 == 1), 1], color = 'b')
plt.scatter(features[(pred_2 == 2), 0], features[(pred_2 == 2), 1], color = 'g')
plt.plot(centers[:, 0], centers[:, 1], 'k+', markersize = 10)
plt.show()

## Clustering Jerárquico: Dendograma

In [ ]:
plt.title('Visualizando la data') 
dendrogram = shc.dendrogram((shc.linkage(X, method ='ward')))

In [ ]:
agg = AgglomerativeClustering(3)
pred = agg.fit_predict(X)

In [ ]:
pred[:10]

In [ ]:
pd.value_counts(pred)

In [ ]:
# tabla de frecuencia relativa
round(100 * pd.value_counts(pred) / len(pred),2)